# 制作干净的数据

In [1]:
import numpy as np
import pandas as pd 
from pathlib import Path
import os 
import shutil
import imghdr
from glob import glob
from tqdm import tqdm 

In [2]:
image_path_df = pd.read_csv("bigdata/temp_dir/temp_image_url.csv")
image_path_df.head(3)

,image_path
0,bigdata/image_data\test-0.jpg
1,bigdata/image_data\test-1.jpg
2,bigdata/image_data\test-10.jpg


In [3]:
train_test_data = pd.concat([pd.read_csv("bigdata/raw_data/test-2.6w.csv"), pd.read_csv("bigdata/raw_data/train-137w.csv")]).reset_index(drop=True)
train_test_data.head(3)

,text,url,filename
0,欧美夏季ebay连衣裙 气质圆领通勤绑带收腰连衣裙 zc3730,https://gimg2.baidu.com/image_search/src=http%...,test-0.jpg
1,"曾是名不见经传的王平,为何能够取代魏延,成为蜀汉",https://pic.rmb.bdstatic.com/19539b3b1a7e1daee...,test-1.jpg
2,女童黄色连衣裙,https://gimg2.baidu.com/image_search/src=http%...,test-2.jpg


In [4]:
clean_train_test_df = image_path_df.pipe(
    lambda x: x.assign(**{
        'filename':x['image_path'].apply(lambda j: Path(j).name)
    })
).pipe(
    lambda x: x.merge(
        right=train_test_data,
        how='left',
        left_on=['filename'],
        right_on=['filename']
    )
)[['image_path', 'text']].pipe(
    lambda x: x.assign(**{
        'train_or_test':x['image_path'].apply(
            lambda j: 'train' if j.find('train') != -1 else 'test'
        )
    })
).pipe(
    lambda x: x.assign(**{
    'image_path':x['image_path'].apply(lambda j : j.replace('\\', '/'))
    })
)

clean_train_test_df.head(3)

,image_path,text,train_or_test
0,bigdata/image_data/test-0.jpg,欧美夏季ebay连衣裙 气质圆领通勤绑带收腰连衣裙 zc3730,test
1,bigdata/image_data/test-1.jpg,"曾是名不见经传的王平,为何能够取代魏延,成为蜀汉",test
2,bigdata/image_data/test-10.jpg,考勤加班明细表,test


In [5]:
clean_train_test_dir = "bigdata/clean_train_test"
if Path(clean_train_test_dir).exists():
    shutil.rmtree(clean_train_test_dir, ignore_errors=True)

os.makedirs(clean_train_test_dir)

In [6]:
clean_train_test_df.query('train_or_test == "test"').drop(columns=['train_or_test']).to_csv(Path(clean_train_test_dir).joinpath("test.csv"), index=False)
clean_train_test_df.query('train_or_test == "train"').drop(columns=['train_or_test']).to_csv(Path(clean_train_test_dir).joinpath("train.csv"), index=False)

In [7]:
clean_train_test_df.query('train_or_test == "test"').drop(columns=['train_or_test']).sample(200).to_csv(Path(clean_train_test_dir).joinpath("test_small.csv"), index=False)
clean_train_test_df.query('train_or_test == "train"').drop(columns=['train_or_test']).sample(1000).to_csv(Path(clean_train_test_dir).joinpath("train_small.csv"), index=False)

In [8]:
clean_train_test_df.query('train_or_test == "train"').sample(20000).pipe(
    lambda x:x['text'].apply(lambda j: len(j))
).median()

24.0

In [9]:
# clean_train_test_df.query('train_or_test == "train"')